# Familiarize with Dataset
Run the following cells to see dataset images with the points used as training data

In [1]:
import json
import numpy as np
import cv2

In [2]:
DATASET_SIZE = 10

with open('data_cleaned.json') as json_file:
    data = json.load(json_file)

# Convert python list to np array
keys_all = sorted(data)
keys_all = np.asarray(keys_all)
rand_indices = np.random.randint(len(data), size=DATASET_SIZE)
keys = keys_all[rand_indices]
print(keys[0])

cebaaf0546e94fcb96c1a186838a73af/positive/cebaaf0546e94fcb96c1a186838a73af_53_480.jpg


Function that returns the required parameters of the solvePnP method. The paramaters *model_points* and *dist_coeffs* are the same for every example so they are declared globally.

In [3]:
def generate_solvepnp_parameters(size, landmarks):
    # Approximate camera intrinsic parameters
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                             [[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )

    # Grab the 2D coordinates of our six sample points
    image_points = np.array([
        (landmarks[33]['x'], landmarks[33]['y']) ,     # Nose tip
        (landmarks[8]['x'], landmarks[8]['y']),     # Chin
        (landmarks[36]['x'], landmarks[36]['y']),     # Left eye left corner
        (landmarks[45]['x'], landmarks[45]['y']),     # Right eye right corner
        (landmarks[48]['x'], landmarks[48]['y']),     # Left Mouth corner
        (landmarks[54]['x'], landmarks[54]['y'])      # Right mouth corner
    ], dtype="double")
    
    return image_points, camera_matrix

# model_points = np.array([
#                             (0.0, 0.0, 0.0),             # Nose tip
#                             (0.0, -330.0, -65.0),        # Chin
#                             (-225.0, 170.0, -135.0),     # Left eye left corner
#                             (225.0, 170.0, -135.0),      # Right eye right corne
#                             (-150.0, -150.0, -125.0),    # Left Mouth corner
#                             (150.0, -150.0, -125.0)      # Right mouth corner
#                         ])

dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion

In [7]:
def get_full_model_points(filename='model_points.txt'):
    """Get all 68 3D model points from file"""
    raw_value = []
    with open(filename) as file:
        for line in file:
            raw_value.append(line)
    model_points = np.array(raw_value, dtype=np.float32)
    model_points = np.reshape(model_points, (3, -1)).T

    # Transform the model into a front view.
    model_points[:, 2] *= -1

    return model_points

In [20]:
model_points = get_full_model_points()

# print the nose 3D model point
print(model_points[33, :])

[ 0.55592   6.5629   25.944448]


In [15]:
def get_full_image_points(landmarks):
    image_points = np.zeros((68, 2))
    
    for i in range(68):
        image_points[i, :] = (landmarks[i]['x'], landmarks[i]['y'])
    
    return image_points

In [16]:
lm = data[keys[0]]['landmarks']
image_points = get_full_image_points(lm)
print(image_points.shape)


(68, 2)


In [22]:
for i in range(DATASET_SIZE):
    key = keys[i]
    im = cv2.imread('dataset/' + key)
    
    # Check if it is positive or negative example
    output = key.split('/')[1]
        
    ex = data[key]
    landmarks = ex['landmarks']
    iris_left = ex['iris_left']
    iris_right = ex['iris_right']
    
    # Change hardcoded size
    size = im.shape
    
    image_points, camera_matrix = generate_solvepnp_parameters(size, landmarks)
    
    image_points = get_full_image_points(landmarks)
    
    # Solve the PnP problem with the parameters specified above
    (suc, rotation_vector, translation_vector) = cv2.solvePnP(
        model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
        )
    
    for point in image_points:
        cv2.circle(im, (int(point[0]), int(point[1])), 3, (0, 0, 255), -1)
    cv2.circle(im, (int(iris_left[0]), int(iris_left[1])), 3, (0, 0, 255), -1)
    cv2.circle(im, (int(iris_right[0]), int(iris_right[1])), 3, (0, 0, 255), -1)
    
    # Project a 3D point (0, 0, 1000.0) onto the image plane.
    # We use this to draw a line sticking out of the nose
    (nose_end_point2D, jacobian) = cv2.projectPoints(
        np.array([(0.55592, 6.5629, 300.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs
        )
    # Draw a line connecting the two points. This line must show
    # the direction out of the nose
    p1 = ( int(image_points[33][0]), int(image_points[33][1]) )
    p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]) )
    cv2.line(im, p1, p2, (255,0,0), 2)
    
    # Display image
    cv2.imshow(output, im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()